In [1]:
import pandas as pd
import numpy as np
import os


import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max.columns',500)

In [4]:
df = pd.read_csv('/Users/roshansk/Documents/PDSG/Projects/ACLU/Final Report/all_counties_unique_otn.csv')

df.drop_duplicates(subset='OTN',inplace = True)

countyList = list(df.county.unique())

/Users/roshansk/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (18,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.head()

,Unnamed: 0,BailAction,BailActionAmount,BailActionAuthority,BailActionCreatedSystem,BailActionDate,BailPostingStatus,BailPostingStatusDate,BailType,Charge,County,CourtNumber,DefendantRace,DefendantSex,Description,DisposingJudge,DistrictCourtNumber,DocketNumber,DocketStart,DocketType,MeanCleanedGravityScore,OTN,OffenseDate,OffenseDisposition,OffenseSequenceNumber,Section,SubSection,SuretyName,SuretyType,System,Title,county,gravity key,hasBail
0,0,Set,0.0,"Leete, John B.",CP-55-53-Crim,1998-08-10 00:00:00,Posted,1998-08-10 00:00:00,ROR,18.3925A,53.0,CP-55-53-Crim,Unknown/Unreported,Male,Receiving Stolen Property,"Minor, Stephen PB",NaN,CP-53-CR-0000145-1998,NaN,CR,6.0,F0428120,1998-08-01 00:00:00,Nolle Prossed,2.0,3925,A,"Groff, Wayne Allen",Self,CPCMS,18,Potter,18.3925A,False
1,1,Set,2500.0,"Easton, Barbara J.",CP-55-53-Crim,2004-07-12 00:00:00,Posted,2004-07-12 15:30:27,Monetary,75.1571A1,53.0,CP-55-53-Crim,Unknown/Unreported,Male,Rec'd Sus'd Fict Alt'd Etc Drivers Lic,"Minor, Stephen PB",NaN,CP-53-CR-0000261-2004,NaN,CR,5.0,E2880275,2002-08-04 00:00:00,Guilty Plea,3.0,1571,A1,"Stumpp, Barry Jr.",Self,CPCMS,75,Potter,75.1571A1,True
2,2,Set,0.0,"Easton, Annette L.",MDJ-55-3-01,2010-10-21 11:13:00,Posted,2010-10-21 09:22:50,ROR,75.3802D1I,53.0,CP-55-53-Crim,White,Male,DUI: Controlled Substance - Schedule 1 - 1st O...,"Minor, Stephen PB",NaN,CP-53-CR-0000200-2010,NaN,CR,4.0,J1646816,2010-08-21 00:00:00,Nolle Prossed,1.0,3802,D1i*,"Breck, Dustin John",Self,CPCMS,75,Potter,75.3802D1I,False
3,3,Set,2500.0,"Stubbs, Kari A.",MDJ-55-4-01,2012-04-24 10:09:00,NaN,NaN,Unsecured,75.3802C,53.0,CP-55-53-Crim,White,Male,DUI: Highest Rte of Alc (BAC .16+) 2nd Off,"Minor, Stephen PB",NaN,CP-53-CR-0000105-2012,NaN,CR,4.0,L6835990,2012-01-20 00:00:00,Guilty Plea,2.0,3802,C**,NaN,NaN,CPCMS,75,Potter,75.3802C,True
4,4,Set,1000.0,"Weiss, Delores G.",MDJ-55-4-03,2012-09-05 15:16:00,NaN,NaN,Monetary,18.3925A,53.0,CP-55-53-Crim,White,Female,Receiving Stolen Property,"Minor, Stephen PB",NaN,CP-53-CR-0000201-2012,NaN,CR,6.0,T2240206,2012-08-15 00:00:00,Guilty Plea,4.0,3925,A,NaN,NaN,CPCMS,18,Potter,18.3925A,True


# Bail Trends

## Cash Bail Rate

### State

In [15]:
val = len(df[df.BailType=='Monetary'])/len(df) 

print(f"Share of Monetary cases : {val}")
print(f"Number of Monetary cases : {len(df[df.BailType=='Monetary'])}")
print(f"Total Number of cases : {len(df)}")

Share of Monetary cases : 0.42596669790166813
Number of Monetary cases : 167741
Total Number of cases : 393789


### County

In [21]:
totalCases = df.groupby('county')['OTN'].count().reset_index()
totalCases.columns = ['county','total_cases']
totalCases.head()

,county,total_cases
0,Adams,2597
1,Allegheny,32286
2,Armstrong,1851
3,Beaver,6275
4,Bedford,1496


In [22]:
cashCases = df[df.BailType=='Monetary'].groupby('county')['OTN'].count().reset_index()
cashCases.columns = ['county','cash_cases']
cashCases.head()

,county,cash_cases
0,Adams,784
1,Allegheny,14047
2,Armstrong,760
3,Beaver,3078
4,Bedford,578


In [24]:
countyDf = pd.merge(totalCases, cashCases, on = 'county')
countyDf

,county,total_cases,cash_cases
0,Adams,2597,784
1,Allegheny,32286,14047
2,Armstrong,1851,760
3,Beaver,6275,3078
4,Bedford,1496,578
...,...,...,...
62,Washington,6829,2659
63,Wayne,801,298
64,Westmoreland,12577,3989
65,Wyoming,1079,385


### MDJ

In [25]:
totalCases = df.groupby('BailActionAuthority')['OTN'].count().reset_index()
totalCases.columns = ['BailActionAuthority','total_cases']
totalCases.head()

,BailActionAuthority,total_cases
0,"Abate, Frank Jr.",4
1,"Abraham, Randy S.",18
2,"Adams, H. Anthony",814
3,"Adams, Joseph C.",2
4,"Aigner, Paula M.",892


In [26]:
cashCases = df[df.BailType=='Monetary'].groupby('BailActionAuthority')['OTN'].count().reset_index()
cashCases.columns = ['BailActionAuthority','cash_cases']
cashCases.head()

,BailActionAuthority,cash_cases
0,"Abate, Frank Jr.",1
1,"Abraham, Randy S.",6
2,"Adams, H. Anthony",206
3,"Adams, Joseph C.",1
4,"Aigner, Paula M.",470


In [29]:
countyDf = pd.merge(totalCases, cashCases, on = 'BailActionAuthority')
countyDf

,BailActionAuthority,total_cases,cash_cases
0,"Abate, Frank Jr.",4,1
1,"Abraham, Randy S.",18,6
2,"Adams, H. Anthony",814,206
3,"Adams, Joseph C.",2,1
4,"Aigner, Paula M.",892,470
...,...,...,...
1048,"Zozos, Paul T.",622,313
1049,"Zucco, Linda",249,74
1050,"Zulick, Arthur L.",14,5
1051,"Zungali, Michael",490,173


## ROR Rate

### State

In [30]:
val = len(df[df.BailType=='ROR'])/len(df) 

print(f"Share of ROR cases : {val}")
print(f"Number of ROR cases : {len(df[df.BailType=='Monetary'])}")
print(f"Total Number of cases : {len(df)}")

Share of ROR cases : 0.2273527193497025
Number of ROR cases : 167741
Total Number of cases : 393789


### County

In [32]:
totalCases = df.groupby('county')['OTN'].count().reset_index()
totalCases.columns = ['county','total_cases']


rorCases = df[df.BailType=='ROR'].groupby('county')['OTN'].count().reset_index()
rorCases.columns = ['county','ror_cases']


outDf = pd.merge(totalCases, rorCases, on = 'county', how='left')
outDf

,county,total_cases,ror_cases
0,Adams,2597,646.0
1,Allegheny,32286,9543.0
2,Armstrong,1851,801.0
3,Beaver,6275,2247.0
4,Bedford,1496,27.0
...,...,...,...
62,Washington,6829,1749.0
63,Wayne,801,277.0
64,Westmoreland,12577,4141.0
65,Wyoming,1079,2.0


### MDJ

In [35]:
totalCases = df.groupby('BailActionAuthority')['OTN'].count().reset_index()
totalCases.columns = ['BailActionAuthority','total_cases']


rorCases = df[df.BailType=='ROR'].groupby('BailActionAuthority')['OTN'].count().reset_index()
rorCases.columns = ['BailActionAuthority','ror_cases']


outDf = pd.merge(totalCases, rorCases, on = 'BailActionAuthority', how='left')
outDf

,BailActionAuthority,total_cases,ror_cases
0,"Abate, Frank Jr.",4,3.0
1,"Abraham, Randy S.",18,NaN
2,"Adams, H. Anthony",814,482.0
3,"Adams, Joseph C.",2,1.0
4,"Aigner, Paula M.",892,NaN
...,...,...,...
1206,"Zucco, Linda",249,99.0
1207,"Zulick, Arthur L.",14,NaN
1208,"Zungali, Michael",490,161.0
1209,"Zupsic, Joseph",1,1.0


## Unsecured Rate

### State

In [36]:
val = len(df[df.BailType=='Unsecured'])/len(df) 

print(f"Share of Unsecured cases : {val}")
print(f"Number of Unsecured cases : {len(df[df.BailType=='Unsecured'])}")
print(f"Total Number of cases : {len(df)}")

Share of Unsecured cases : 0.2929335253143181
Number of Unsecured cases : 115354
Total Number of cases : 393789


### County

In [38]:
totalCases = df.groupby('county')['OTN'].count().reset_index()
totalCases.columns = ['county','total_cases']


unsecuredCases = df[df.BailType=='Unsecured'].groupby('county')['OTN'].count().reset_index()
unsecuredCases.columns = ['county','unsecured_cases']


outDf = pd.merge(totalCases, unsecuredCases, on = 'county', how='left')
outDf

,county,total_cases,unsecured_cases
0,Adams,2597,790
1,Allegheny,32286,894
2,Armstrong,1851,229
3,Beaver,6275,794
4,Bedford,1496,891
...,...,...,...
62,Washington,6829,2335
63,Wayne,801,226
64,Westmoreland,12577,3859
65,Wyoming,1079,689


### MDJ

In [40]:
totalCases = df.groupby('BailActionAuthority')['OTN'].count().reset_index()
totalCases.columns = ['BailActionAuthority','total_cases']


unsecuredCases = df[df.BailType=='Unsecured'].groupby('BailActionAuthority')['OTN'].count().reset_index()
unsecuredCases.columns = ['BailActionAuthority','unsecured_cases']


outDf = pd.merge(totalCases, unsecuredCases, on = 'BailActionAuthority', how='left')
outDf

,BailActionAuthority,total_cases,unsecured_cases
0,"Abate, Frank Jr.",4,NaN
1,"Abraham, Randy S.",18,12.0
2,"Adams, H. Anthony",814,125.0
3,"Adams, Joseph C.",2,NaN
4,"Aigner, Paula M.",892,422.0
...,...,...,...
1206,"Zucco, Linda",249,NaN
1207,"Zulick, Arthur L.",14,9.0
1208,"Zungali, Michael",490,155.0
1209,"Zupsic, Joseph",1,NaN


## Nonmonetary

### State

In [42]:
val = len(df[df.BailType=='Nonmonetary'])/len(df) 

print(f"Share of Nonmonetary cases : {val}")
print(f"Number of Nonmonetary cases : {len(df[df.BailType=='Nonmonetary'])}")
print(f"Total Number of cases : {len(df)}")

Share of Nonmonetary cases : 0.04791652382367207
Number of Nonmonetary cases : 18869
Total Number of cases : 393789


### County

In [43]:
totalCases = df.groupby('county')['OTN'].count().reset_index()
totalCases.columns = ['county','total_cases']


unsecuredCases = df[df.BailType=='Nonmonetary'].groupby('county')['OTN'].count().reset_index()
unsecuredCases.columns = ['county','nonmonetary_cases']


outDf = pd.merge(totalCases, unsecuredCases, on = 'county', how='left')
outDf

,county,total_cases,nonmonetary_cases
0,Adams,2597,372.0
1,Allegheny,32286,7683.0
2,Armstrong,1851,61.0
3,Beaver,6275,115.0
4,Bedford,1496,NaN
...,...,...,...
62,Washington,6829,65.0
63,Wayne,801,NaN
64,Westmoreland,12577,586.0
65,Wyoming,1079,1.0


### MDJ

In [44]:
totalCases = df.groupby('BailActionAuthority')['OTN'].count().reset_index()
totalCases.columns = ['BailActionAuthority','total_cases']


unsecuredCases = df[df.BailType=='Nonmonetary'].groupby('BailActionAuthority')['OTN'].count().reset_index()
unsecuredCases.columns = ['BailActionAuthority','nonmonetary_cases']


outDf = pd.merge(totalCases, unsecuredCases, on = 'BailActionAuthority', how='left')
outDf

,BailActionAuthority,total_cases,nonmonetary_cases
0,"Abate, Frank Jr.",4,NaN
1,"Abraham, Randy S.",18,NaN
2,"Adams, H. Anthony",814,NaN
3,"Adams, Joseph C.",2,NaN
4,"Aigner, Paula M.",892,NaN
...,...,...,...
1206,"Zucco, Linda",249,75.0
1207,"Zulick, Arthur L.",14,NaN
1208,"Zungali, Michael",490,NaN
1209,"Zupsic, Joseph",1,NaN


## Mean Bail Amount

### State

In [51]:
out  = df[df.BailType.isin(['Monetary','Nominal','Nonmonetary','Unsecured','ROR'])].groupby('BailType').agg({'BailActionAmount':'mean','OTN':'count'}).reset_index()
out.columns = ['BailType','meanBailAmount','numCases']
out


,BailType,meanBailAmount,numCases
0,Monetary,47254.570275,167740
1,Nominal,2903.742963,513
2,Nonmonetary,0.000000,18869
3,ROR,0.000034,89529
4,Unsecured,8316.749841,115354


#### After removing outliers (99th percentile)

In [76]:
bailTypes = ['Monetary','Nominal','Unsecured']

for i in range(len(bailTypes)):
    sub = df[df.BailType==bailTypes[i]]
    
    percentValue = np.quantile(sub.BailActionAmount,0.99)
    
    meanBailAmount = np.mean(sub[sub.BailActionAmount<=percentValue]['BailActionAmount'])
    
    print(f"{bailTypes[i]:10} : {meanBailAmount}")
    


Monetary   : 35949.67122303108
Nominal    : 817.7912031558186
Unsecured  : 7289.618016347391


### County

In [67]:
out  = df[df.BailType.isin(['Monetary','Nominal','Nonmonetary','Unsecured','ROR'])].groupby(['county','BailType']).agg({'BailActionAmount':'mean','OTN':'count'}).reset_index()
out.columns = ['county','BailType','meanBailAmount','numCases']
out

,county,BailType,meanBailAmount,numCases
0,Adams,Monetary,34094.387755,784
1,Adams,Nonmonetary,0.000000,372
2,Adams,ROR,0.000000,646
3,Adams,Unsecured,6116.455696,790
4,Allegheny,Monetary,21868.048799,14047
...,...,...,...,...
277,York,Monetary,28995.485566,7438
278,York,Nominal,2690.586207,58
279,York,Nonmonetary,0.000000,959
280,York,ROR,0.000000,5596


#### After removing outliers (99th percentile)

In [75]:
countyList = list(df.county.unique())

bailTypes = ['Monetary','Nominal','Unsecured']


for j in range(len(countyList)):
    for i in range(len(bailTypes)):
        sub = df[(df.county == countyList[j]) & (df.BailType==bailTypes[i])]
        
        if len(sub)==0:
            continue

        percentValue = np.quantile(sub.BailActionAmount,0.99)

        meanBailAmount = np.mean(sub[sub.BailActionAmount<=percentValue]['BailActionAmount'])

        print(f"{countyList[j]:15} {bailTypes[i]:10} : {meanBailAmount}")



Potter          Monetary   : 22323.943661971833
Potter          Unsecured  : 7808.781869688385
Cumberland      Monetary   : 28436.02593440122
Cumberland      Nominal    : 1.0
Cumberland      Unsecured  : 6468.6579150579155
Pike            Monetary   : 28077.22900763359
Pike            Nominal    : 10000.0
Pike            Unsecured  : 9409.23076923077
Forest          Monetary   : 27210.526315789473
Forest          Unsecured  : 18255.81395348837
Wayne           Monetary   : 34180.0
Wayne           Unsecured  : 8535.714285714286
Fulton          Monetary   : 54747.899159663866
Fulton          Unsecured  : 32983.87096774193
Lackawanna      Monetary   : 31853.256777812403
Lackawanna      Nominal    : 1357.07
Lackawanna      Unsecured  : 10281.341390814558
Dauphin         Monetary   : 40805.15511771581
Dauphin         Nominal    : 9.285714285714286
Dauphin         Unsecured  : 7140.254872563718
Huntingdon      Monetary   : 20545.026642984016
Huntingdon      Unsecured  : 8332.55269320843
Somer

## % of MDJs who use cash bail more than any other bail type

In [102]:
out = df.groupby(['BailActionAuthority','BailType'])['OTN'].count().reset_index()
out = out.pivot(index='BailActionAuthority', columns='BailType', values='OTN').reset_index()
out

BailType,BailActionAuthority,Migrated Bail Type,Monetary,Monetary - Cash Percentage,Nominal,Nonmonetary,ROR,ROR - Common Pleas,Unsecured
0,"Abate, Frank Jr.",NaN,1.0,NaN,NaN,NaN,3.0,NaN,NaN
1,"Abraham, Randy S.",NaN,6.0,NaN,NaN,NaN,NaN,NaN,12.0
2,"Adams, H. Anthony",NaN,206.0,NaN,NaN,NaN,482.0,NaN,125.0
3,"Adams, Joseph C.",NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN
4,"Aigner, Paula M.",NaN,470.0,NaN,NaN,NaN,NaN,NaN,422.0
...,...,...,...,...,...,...,...,...,...
1205,"Zucco, Linda",NaN,74.0,NaN,NaN,75.0,99.0,NaN,NaN
1206,"Zulick, Arthur L.",NaN,5.0,NaN,NaN,NaN,NaN,NaN,9.0
1207,"Zungali, Michael",NaN,173.0,NaN,NaN,NaN,161.0,NaN,155.0
1208,"Zupsic, Joseph",NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [103]:
out['mostCommonBailType'] = out[out.columns[1:]].apply(lambda x : np.argmax(x), axis = 1)

In [104]:
out.head()

BailType,BailActionAuthority,Migrated Bail Type,Monetary,Monetary - Cash Percentage,Nominal,Nonmonetary,ROR,ROR - Common Pleas,Unsecured,mostCommonBailType
0,"Abate, Frank Jr.",NaN,1.0,NaN,NaN,NaN,3.0,NaN,NaN,ROR
1,"Abraham, Randy S.",NaN,6.0,NaN,NaN,NaN,NaN,NaN,12.0,Unsecured
2,"Adams, H. Anthony",NaN,206.0,NaN,NaN,NaN,482.0,NaN,125.0,ROR
3,"Adams, Joseph C.",NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,Monetary
4,"Aigner, Paula M.",NaN,470.0,NaN,NaN,NaN,NaN,NaN,422.0,Monetary


In [108]:
### Most common bail types for a given judge

# For 38.18% of all MDJs, Unsecured is the most common type of bail that they assign
# For 37.85% of all MDJs, Monetary is the most common type of bail that they assign

out.mostCommonBailType.value_counts()/len(out)

Unsecured                     0.381818
Monetary                      0.378512
ROR                           0.189256
Nonmonetary                   0.030579
ROR - Common Pleas            0.008264
Nominal                       0.006612
Monetary - Cash Percentage    0.004132
Migrated Bail Type            0.000826
Name: mostCommonBailType, dtype: float64

## Bail Posting Rate

### State

In [112]:
### Overall Bail posting status
df.BailPostingStatus.value_counts(dropna=False)

NaN                257854
Posted             132838
Partial Posting      1659
Modified             1438
Name: BailPostingStatus, dtype: int64

In [113]:
### Bail Posting status for cash bail (monetary) cases
df[df.BailType == 'Monetary'].BailPostingStatus.value_counts(dropna=False)

NaN                98164
Posted             67952
Partial Posting     1367
Modified             258
Name: BailPostingStatus, dtype: int64

### County

In [126]:
### County wise Bail posting status
sub = df.copy()

# 'blank' represents cases where there was no record for BailPostingStatus
sub.BailPostingStatus.fillna('blank',inplace = True)

out = sub.groupby(['county','BailPostingStatus']).OTN.count().reset_index()
out.columns = ['county','BailStatus','numCases']
out

,county,BailStatus,numCases
0,Adams,Modified,5
1,Adams,Partial Posting,1
2,Adams,Posted,2169
3,Adams,blank,422
4,Allegheny,Modified,10
...,...,...,...
210,Wyoming,blank,948
211,York,Modified,4
212,York,Partial Posting,13
213,York,Posted,4131


# Charges

In [137]:
### Most common crimes associated with cash bail
out = df[df.BailType=='Monetary']['Description'].value_counts().reset_index()
out.columns = ['Description','numCases']
out.head(10)

,Description,numCases
0,"Manufacture, Delivery, or Possession With Inte...",26495
1,Receiving Stolen Property,13545
2,Aggravated Assault,13128
3,Simple Assault,11295
4,Int Poss Contr Subst By Per Not Reg,8866
5,Retail Theft-Take Mdse,8417
6,Terroristic Threats W/ Int To Terrorize Another,5325
7,Firearms Not To Be Carried W/O License,4042
8,Theft By Unlaw Taking-Movable Prop,3129
9,Use/Poss Of Drug Paraph,2728


In [136]:
### Share of monetary cases for these top crimes. What percentage of these crimes are set monetary bail


for i in range(10):
    crime = out.Description.iloc[i]
    cashCases = out.numCases.iloc[i]
    
    totalCases = len(df[df.Description==crime])
    
    print(f"{crime} : {cashCases/totalCases}")

Manufacture, Delivery, or Possession With Intent to Manufacture or Deliver : 0.7909899689515166
Receiving Stolen Property : 0.4678433268858801
Aggravated Assault : 0.8554114810712191
Simple Assault : 0.5296600234466589
Int Poss Contr Subst By Per Not Reg : 0.30236682354546074
Retail Theft-Take Mdse : 0.38571166712491983
Terroristic Threats W/ Int To Terrorize Another : 0.6659579789894947
Firearms Not To Be Carried W/O License : 0.9002227171492205
Theft By Unlaw Taking-Movable Prop : 0.41350601295097134
Use/Poss Of Drug Paraph : 0.2896273489754751


# Cash Bail and Race

In [138]:
### Function to retreive cash bail rate
def getCashBailRate(x):
    
    cash = len(x[x.BailType == 'Monetary'])
    
    return cash/len(x)

## Cash bail rate for Black v. white defendants

### State

In [139]:
white = df[df.DefendantRace=='White']
black = df[df.DefendantRace=='Black']

whiteCashRate = getCashBailRate(white)
blackCashRate = getCashBailRate(black)

print(f"Cash Bail Rates | White : {whiteCashRate}  Black : {blackCashRate}")

Cash Bail Rates | White : 0.3787542235602311  Black : 0.5474625249906149


### County

In [172]:
print("{:15} {:<12} {:<11} {:<11} {:<10}".format('county','whiteRate','blackRate','whiteCases','blackCases'))
for i in range(len(countyList)):
    
    whiteRate = np.round(getCashBailRate(white[white.county==countyList[i]]),3)
    blackRate = np.round(getCashBailRate(black[black.county==countyList[i]]),3)
    
    whiteCases = len(white[white.county==countyList[i]])
    blackCases = len(black[black.county==countyList[i]])
    
    
    print(f"{countyList[i]:15} {whiteRate :<10}   {blackRate:<10}  {whiteCases:<10}  {blackCases:<10}")

county          whiteRate    blackRate   whiteCases  blackCases
Potter          0.232        0.75        585         8         
Cumberland      0.35         0.516       5310        1422      
Pike            0.34         0.374       970         139       
Forest          0.32         0.85        103         20        
Wayne           0.353        0.649       742         37        
Fulton          0.293        0.316       345         38        
Lackawanna      0.49         0.671       4572        1211      
Dauphin         0.365        0.512       7505        6080      
Huntingdon      0.378        0.641       1200        153       
Somerset        0.232        0.696       1870        92        
Cameron         0.255        1.0         110         2         
Montgomery      0.287        0.433       12367       7335      
Tioga           0.345        0.57        2185        200       
Washington      0.358        0.558       5480        1135      
Bedford         0.376        0.526      

### MDJ

In [176]:
mdjList = list(df.BailActionAuthority.unique())

print("{:30} {:<12} {:<11} {:<11} {:<10}".format('MDJ','whiteRate','blackRate','whiteCases','blackCases'))
for i in range(len(mdjList)):
    
    whiteCases = len(white[white.BailActionAuthority==mdjList[i]])
    blackCases = len(black[black.BailActionAuthority==mdjList[i]])
    
    
    if whiteCases == 0:
        whiteRate = 0
    else:
        whiteRate = np.round(getCashBailRate(white[white.BailActionAuthority==mdjList[i]]),3)
        
    if blackCases == 0:
        blackRate = 0
    else:
        blackRate = np.round(getCashBailRate(black[black.BailActionAuthority==mdjList[i]]),3)
    
    
    
    print(f"{mdjList[i]:30} {whiteRate :<10}   {blackRate:<10}  {whiteCases:<10}  {blackCases:<10}")

MDJ                            whiteRate    blackRate   whiteCases  blackCases
Leete, John B.                 0.5          0           2           0         
Easton, Barbara J.             0            0           0           0         
Easton, Annette L.             0.186        1.0         231         4         
Stubbs, Kari A.                0.0          0           1           0         
Weiss, Delores G.              0.247        0           182         0         
McCleaft, Kari A.              0.275        0.5         160         4         
                           nan 0            0           0           0         
Minor, Stephen PB              0.375        0           8           0         
Clement, Charles A.            0.38         0.593       498         150       
Day, Susan K.                  0.387        0.509       493         114       
Martin, Mark W.                0.317        0.5         565         122       
Manlove, Robert V.             1.0          1.0     

## Mean Bail Amount for Black v. White defendants

In [177]:
def getMeanBail(x):
    
    df = x.copy()
    quant = np.quantile(df.BailActionAmount, 0.99) #Removing outliers
    df = df[df.BailActionAmount<quant]
    
    return np.mean(df.BailActionAmount)

In [179]:
sub = df[df.BailType=='Monetary']
countyList = list(df.county.unique())

black = sub[sub.DefendantRace=='Black']
white = sub[sub.DefendantRace=='White']

### State

In [180]:
getMeanBail(black)

38700.61967697773

In [181]:
getMeanBail(white)

27572.555135439507

### County 

In [187]:
for i in range(len(countyList)):
    
    whiteBailAmount = np.round(getMeanBail(white[white.county==countyList[i]]),3)
    blackBailAmount = np.round(getMeanBail(black[black.county==countyList[i]]),3)
      
    
    print(f"{countyList[i]:15} {whiteBailAmount:<10} {blackBailAmount:<10}")

Potter          19338.462  15700.0   
Cumberland      23280.564  34036.613 
Pike            21534.016  34754.902 
Forest          22343.75   21437.5   
Wayne           32100.772  28681.818 
Fulton          57878.788  24000.0   
Lackawanna      26940.471  41103.899 
Dauphin         32312.922  40031.238 
Huntingdon      18028.132  22010.309 
Somerset        33896.27   78071.429 
Cameron         26481.481  10000.0   
Montgomery      21885.175  29168.796 
Tioga           18916.287  30454.955 
Washington      19877.786  33526.861 
Bedford         49175.889  65010.0   
Susquehanna     27137.793  29807.692 
Erie            23611.419  38881.395 
Bucks           58683.401  73634.626 
Carbon          24452.426  35236.275 
Bradford        32077.371  63125.0   
Fayette         17203.453  25443.667 
Jefferson       45706.776  35040.0   
Schuylkill      19996.062  28947.368 
Northumberland  38641.424  63255.285 
Monroe          21494.509  34085.027 
Lawrence        15192.057  25778.481 
Delaware    

## % of criminal cases that are Black v. white

### State

In [188]:
df.DefendantRace.value_counts()/len(df)

White                               0.675659
Black                               0.290874
Unknown/Unreported                  0.012865
Asian/Pacific Islander              0.003850
Asian                               0.003705
Native American/Alaskan Native      0.000739
Bi-Racial                           0.000460
Native Hawaiian/Pacific Islander    0.000030
Name: DefendantRace, dtype: float64

### County

In [216]:
out = df[df.DefendantRace.isin(['Black','White'])].groupby(['county','DefendantRace'])['OTN'].count().reset_index()
out = out.pivot(index='county',columns='DefendantRace', values='OTN').reset_index()
out.columns.name = None
out.columns = ['county','blackCases','whiteCases']



totalCases = df.county.value_counts().reset_index()
totalCases.columns = ['county','totalCases']

out = out.merge(totalCases, on ='county',how='left')

out['blackCases']  = out.blackCases/out.totalCases
out['whiteCases']  = out.whiteCases/out.totalCases

In [217]:
# 87% of the cases in Adams are made up of white defendants, 10% by black defendants
out

,county,blackCases,whiteCases,totalCases
0,Adams,0.101656,0.875241,2597
1,Allegheny,0.382209,0.600353,32286
2,Armstrong,0.063749,0.913020,1851
3,Beaver,0.279681,0.671235,6275
4,Bedford,0.064840,0.911096,1496
...,...,...,...,...
62,Washington,0.166203,0.802460,6829
63,Wayne,0.046192,0.926342,801
64,Westmoreland,0.157589,0.791286,12577
65,Wyoming,0.023170,0.955514,1079


## % of cash bail cases that are Black v. white

### State

In [212]:
# Of all the monetary cases, 60% of them have white defendants and 37% have black defendants
sub = df[df.BailType == 'Monetary']

sub.DefendantRace.value_counts()/len(sub)

White                               0.600771
Black                               0.373838
Unknown/Unreported                  0.009694
Asian/Pacific Islander              0.003803
Asian                               0.002319
Native American/Alaskan Native      0.000668
Bi-Racial                           0.000376
Native Hawaiian/Pacific Islander    0.000006
Name: DefendantRace, dtype: float64

### County

In [213]:

out = sub[sub.DefendantRace.isin(['Black','White'])].groupby(['county','DefendantRace'])['OTN'].count().reset_index()
out = out.pivot(index='county',columns='DefendantRace', values='OTN').reset_index()
out.columns.name = None
out.columns = ['county','blackCases','whiteCases']



totalCases = sub.county.value_counts().reset_index()
totalCases.columns = ['county','totalCases']

out = out.merge(totalCases, on ='county',how='left')

out['blackCases']  = out.blackCases/out.totalCases
out['whiteCases']  = out.whiteCases/out.totalCases

In [214]:
out

,county,blackCases,whiteCases,totalCases
0,Adams,0.146684,0.841837,784
1,Allegheny,0.501317,0.487079,14047
2,Armstrong,0.097368,0.878947,760
3,Beaver,0.366797,0.601040,3078
4,Bedford,0.088235,0.885813,578
...,...,...,...,...
62,Washington,0.238059,0.738247,2659
63,Wayne,0.080537,0.879195,298
64,Westmoreland,0.248684,0.713963,3989
65,Wyoming,0.025974,0.953247,385


# Identifying Outliers

### Outlier Counties

In [218]:
out = df[df.DefendantRace.isin(['Black','White'])].groupby(['county','DefendantRace'])['OTN'].count().reset_index()
out = out.pivot(index='county',columns='DefendantRace', values='OTN').reset_index()
out.columns.name = None
out.columns = ['county','blackCases','whiteCases']

In [219]:
out

,county,blackCases,whiteCases
0,Adams,264,2273
1,Allegheny,12340,19383
2,Armstrong,118,1690
3,Beaver,1755,4212
4,Bedford,97,1363
...,...,...,...
62,Washington,1135,5480
63,Wayne,37,742
64,Westmoreland,1982,9952
65,Wyoming,25,1031


In [222]:
print(f"{len(out[out.blackCases<50])} counties where there are less than 50 cases associated with black defendants")

out[out.blackCases<50]

15 counties where there are less than 50 cases associated with black defendants


,county,blackCases,whiteCases
11,Cameron,2,110
19,Crawford,30,738
23,Elk,18,781
26,Forest,20,103
28,Fulton,38,345
29,Greene,39,795
32,Jefferson,37,1129
33,Juniata,16,439
46,Montour,26,249
52,Potter,8,585


### Outlier MDJs

In [223]:
out = df[df.DefendantRace.isin(['Black','White'])].groupby(['BailActionAuthority','DefendantRace'])['OTN'].count().reset_index()
out = out.pivot(index='BailActionAuthority',columns='DefendantRace', values='OTN').reset_index()
out.columns.name = None
out.columns = ['BailActionAuthority','blackCases','whiteCases']

In [224]:
out

,BailActionAuthority,blackCases,whiteCases
0,"Abate, Frank Jr.",1.0,3.0
1,"Abraham, Randy S.",2.0,16.0
2,"Adams, H. Anthony",158.0,644.0
3,"Adams, Joseph C.",1.0,1.0
4,"Aigner, Paula M.",112.0,765.0
...,...,...,...
1198,"Zucco, Linda",81.0,165.0
1199,"Zulick, Arthur L.",6.0,7.0
1200,"Zungali, Michael",42.0,440.0
1201,"Zupsic, Joseph",NaN,1.0


In [228]:
print(f"{len(out[out.blackCases<30])} MDJs out of total {len(out)} have less than 30 cases associated with black defendants")

out[out.blackCases<30]

510 MDJs out of total 1203 have less than 30 cases associated with black defendants


,BailActionAuthority,blackCases,whiteCases
0,"Abate, Frank Jr.",1.0,3.0
1,"Abraham, Randy S.",2.0,16.0
3,"Adams, Joseph C.",1.0,1.0
8,"Albright, Kent H.",1.0,1.0
9,Allegheny Arraignment Court MDJ 05-0-04,11.0,13.0
...,...,...,...
1191,"Zito, Leonard N.",3.0,3.0
1193,"Zoller, Richard H.",3.0,2.0
1194,"Zook, Jeremiah D.",15.0,35.0
1195,"Zottola, John A.",1.0,NaN
